# SoLU Circuits

## Imports

In [1]:
import collections
import copy
import gc
import html
import itertools
import json
import math
import os
import pickle
import random
import sys
import time
from functools import partial
from os import path
from pathlib import Path
from pprint import pprint

import datasets
import einops
import gdown
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm.auto as tqdm
import tqdm.notebook as tqdm
import transformers
import wandb
from datasets import load_dataset
from easy_transformer.EasyTransformer import (MLP, Attention, EasyTransformer,
                                              Embed, LayerNorm, PosEmbed,
                                              TransformerBlock, Unembed)
from easy_transformer.EasyTransformerConfig import EasyTransformerConfig
from easy_transformer.hook_points import HookedRootModule, HookPoint
from IPython.core.display import HTML
from IPython.display import clear_output, display
from rich import print
from torch.utils.data import DataLoader
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# Fix for pysvelte import bug
sys.path.append('/workspaces/solu-circuits/PySvelte')

In [2]:
# Run this after the above fix
import pysvelte

## Model

### Config

Given that we're using a checkpoint of a model that has already been run, we add in the config settings from that model here.

In [3]:
# EasyTransformerConfig settings
cfg = {
    'd_model': 1024,
    'd_head': 64,
    'n_layers': 1,
    'n_ctx': 1024,
    'd_vocab': 50278,
    'use_attn_result': False,
    'act_fn': 'solu_ln',
    'eps': 1e-5,
    # The trained model used LN everywhere, except for RMS just before 
    # the final unembedding. We switch to LNPre (folding in the weights/biases
    # to the next weights), and then manually override the final RMS normalization
    # to be RMSPre in the code below. See the 'Fold in weights and biases' section
    # for more details.
    'normalization_type': 'LNPre',
    "model_name": "SoLU"
}

# Calculated settings
cfg['n_heads'] = cfg['d_model']//cfg['d_head']
cfg['d_mlp'] = 4 * cfg['d_model']

# Custom settings not supported by EasyTransformer directly
custom_cfg = {
    'model_checkpoint_name': 'SoLU_1L_1024W_final_checkpoint.pth',
    'device': 'cuda',
}

### Model Setup

This uses the `EasyTransformer` components, where possible (as they can be configured identically to the code that was used for training).

In [4]:
class RMSNormPre(nn.Module):
    """RMS Pre Normalization
    
    This is RMS Normalization without the multiplation by a weights term, 
    as that has been folded into the next layer's weights instead."""

    def __init__(self, cfg, length):
        super().__init__()
        self.eps = cfg.eps
        self.length = length
        # self.w = nn.Parameter(torch.ones(length)) # Folded

        # Adds a hook point for the normalization scale factor
        self.hook_scale = HookPoint()  # [batch, pos]

    def forward(self, x):
        scale = self.hook_scale((x.pow(2).mean(-1, keepdim=True) +
                                 self.eps).sqrt())  # [batch, pos, 1]
        out = (x / scale) # * self.w # (folded)
        return out

    
class Transformer(EasyTransformer):
    """Transformer
    
    The checkpointed model had a few modifications from the
    standard `EasyTransformer`, so we extend it and add these in here.
    """
    def __init__(self, cfg: EasyTransformerConfig):
        super().__init__("custom", cfg=cfg)
        
        # Custom tokenizer setup (different pad token) from trained model
        self.tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')
        pad_token = '<PAD>'
        self.tokenizer.add_special_tokens({'pad_token': pad_token})
        
        # Custom final layer norm (trained model used RMS Norm, and we've 
        # folded the weights out of this)
        self.ln_final = RMSNormPre(self.cfg, self.cfg.d_model)
   
    def to_tokens(self, text):
        return self.tokenizer(self.tokenizer.bos_token+text, return_tensors='pt')['input_ids'].to(custom_cfg['device'])

    
# Create the model
model = Transformer(EasyTransformerConfig.from_dict(cfg))
model.to(custom_cfg['device'])

Transformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_attn): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
        (hook_post_ln): HookPoint()
        (ln): LayerNorm(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
      )
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hoo

## Load from the checkpoint

### Download Checkpoint

In [5]:
# Checkpoint provided by Neel Nanda
checkpoint_url = "https://drive.google.com/file/d/16bqEZg9Oq0WT2xOcNS1HJkmR7qB2G14o/view"

# Create the directory if it doesn't exist
checkpoint_dir = "/tmp/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Download the checkpoint if it doesn't exist
checkpoint_file = path.join(checkpoint_dir, custom_cfg['model_checkpoint_name'])
if not path.exists(checkpoint_file):
    gdown.download(checkpoint_url, checkpoint_file, quiet=False, fuzzy=True)

### Fold in weights and biases

We fold the `LayerNorm` weights and biases in to the weights after them, for simplicty, as per [A Mathematical Framework for Transformer Circuits](https://transformer-circuits.pub/2021/framework/index.html#model-simplifications).

In [6]:
# Get the state dictionary from the checkpoint
sd = torch.load(checkpoint_file)
print(sd.keys())

# Fold in layer normalization weights & biases, for each layer (just one in the toy example)
for layer in range(cfg['n_layers']):
    # Pre-attention layer norm weights -> Query/Key/Value weights
    pre_ln_w = sd[f"blocks.{layer}.norm1.w"]
    W_Q_old = sd[f"blocks.{layer}.attn.W_Q"]
    W_K_old = sd[f"blocks.{layer}.attn.W_K"]
    W_V_old = sd[f"blocks.{layer}.attn.W_V"]
    sd[f"blocks.{layer}.attn.W_Q"] = W_Q_old * pre_ln_w
    sd[f"blocks.{layer}.attn.W_K"] = W_K_old * pre_ln_w
    sd[f"blocks.{layer}.attn.W_V"] = W_V_old * pre_ln_w
    
    # Pre-attention layer norm biases -> Query/Key/Value biases
    pre_ln_b = sd[f"blocks.{layer}.norm1.b"]
    sd[f"blocks.{layer}.attn.b_Q"] = W_Q_old @ pre_ln_b + sd[f"blocks.{layer}.attn.b_Q"]
    sd[f"blocks.{layer}.attn.b_K"] = W_K_old @ pre_ln_b + sd[f"blocks.{layer}.attn.b_K"]
    sd[f"blocks.{layer}.attn.b_V"] = W_V_old @ pre_ln_b + sd[f"blocks.{layer}.attn.b_V"]
    
    # Post-attention layer weights/biases -> MLP weights/biases
    W_in_old = sd[f"blocks.{layer}.mlp.W_in"]
    sd[f"blocks.{layer}.mlp.W_in"] = W_in_old * sd[f"blocks.{layer}.norm2.w"]
    sd[f"blocks.{layer}.mlp.b_in"] = W_in_old @ sd[f"blocks.{layer}.norm2.b"] \
                                        + sd[f"blocks.{layer}.mlp.b_in"]
    
    # Delete the weights/biases that are no longer used (as they're folded in)
    del sd[f"blocks.{layer}.norm1.w"]
    del sd[f"blocks.{layer}.norm1.b"]
    del sd[f"blocks.{layer}.norm2.w"]
    del sd[f"blocks.{layer}.norm2.b"]

# Fold the post-blocks (pre-unembed) RMS norm weights -> unembed weights
sd["unembed.W_U"] *= sd["norm.w"]
del sd["norm.w"] # Delete as no longer used (folded in)
    
# EasyTransformer has an additional bias term for the unembedding, so we simply set it to zero.
sd["unembed.b_U"] = torch.zeros(cfg['d_vocab'])

# Load the state dict into the model
model.load_state_dict(sd)

odict_keys(['embed.W_E', 'pos_embed.W_pos', 'norm.w', 'blocks.0.norm1.w', 'blocks.0.norm1.b', 'blocks.0.norm2.w', 
'blocks.0.norm2.b', 'blocks.0.attn.W_Q', 'blocks.0.attn.b_Q', 'blocks.0.attn.W_K', 'blocks.0.attn.b_K', 
'blocks.0.attn.W_V', 'blocks.0.attn.b_V', 'blocks.0.attn.W_O', 'blocks.0.attn.b_O', 'blocks.0.attn.mask', 
'blocks.0.attn.IGNORE', 'blocks.0.mlp.W_in', 'blocks.0.mlp.b_in', 'blocks.0.mlp.W_out', 'blocks.0.mlp.b_out', 
'blocks.0.mlp.ln.w', 'blocks.0.mlp.ln.b', 'unembed.W_U'])

<All keys matched successfully>

## Find interesting activations

A 1-layer model without an MLP can't do much more than skip trigrams. Whilst the MLP layer added may improve this a little, the prompts will still need to have quite simple answers.

In this case we'll look for the ability of the model to close HTML tags. As an simple overview of how HTML tags work, whenever a tag is used (e.g. `<b>` for bold) it must be closed when you no longer want it to apply (e.g. `<b>bold text</b> normal text`).

Note that `</` is a single token - so we can't use `<` as the last token and expect to see `/`.

In [73]:
def get_next_token(prompt: str) -> (str, float):
    """Run a forward pass to get the next token"""
    # Get all predictions
    logits = model(prompt, "logits")
    predictions = torch.argmax(logits, 2)
    probabilities = F.softmax(logits, dim=-1)
    
    # Get the prediction for the next token
    prediction = predictions[0][-1].item()
    probability = probabilities[0][-1][prediction].item()
    next_token = model.tokenizer.decode(prediction)
    
    return (next_token, probability)

# Example prompts to run through the model
prompts = [
    "<h1>Title</",
    "<b>Some bold text</",
    "<p>An interesting paragraph</",
    "<table><tr><th>Model name</",
    "<tr><td>Table cell</",
    "<li>List item</",
    "<ul><li>Bullet point</",
    "<div><img src='icon.jpg' /></", # Check when there is a self-closing tag in-between
    "<fake>Fake tag</", # Check for a fake html tag (note the model could also parse generic XML, so it may work)
    "<aowfdyekjds>Something</", # Another fake tag (probably not in the data
]

# Run each prompt (with a few tokens appended by the model)
for prompt in prompts:
    next_token, probability = get_next_token(prompt)
    print(prompt + next_token, str(round(probability * 100, 1)) + "%")

<h1>Title</h 83.6%

<b>Some bold text</b 96.0%

<p>An interesting paragraph</p 95.2%

<table><tr><th>Model name</th 97.9%

<tr><td>Table cell</td 99.7%

<li>List item</li 98.9%

<ul><li>Bullet point</li 82.5%

<div><img src='icon.jpg' /></div 85.4%

<fake>Fake tag</Fake 14.2%

<c>Something</c 79.3%

The last one is interesting, as it suggests the model may have learnt specific html tags rather than just a general pattern.

## Find Important Neurons

### MLP neurons

The MLP activations are multiplied by $W_\text{out}$ (and added to $b_\text{out}$), added to the residual stream & sent through the RMS activation with weights/biases folded out (i.e. removed). It's then then multiplied by the unembedding weights ($W_U$) to get the logits. This means we can calculate the importance of the neurons by multiplying these weights by the neuron values.

In [8]:
prompts_neuron_importance: dict[str, np.ndarray] = {} # {prompt: [importance of each neuron]}

# Get the importance of each neuron, for the list of prompts
for prompt in prompts:
    # Setup the cache
    cache = {}
    model.cache_all(cache)

    # Get the logits
    logits = model(prompt, "logits")[0] # First batch item -> [ tokens x d_vocab ]
    predictions = torch.argmax(logits, 1) # [ tokens ]

    # Get the neurons
    mlp_neurons = cache[f'blocks.{layer}.mlp.hook_post_ln'][0] # [ d_vocab x d_model ]

    # Get the combined weights these are multiplied by
    mlp_neuron_weights = sd['unembed.W_U'] @ sd['blocks.0.mlp.W_out'] # [ d_vocab x d_model ]

    # Get the values for just the last token (the one we've predicted)
    last_token_idx = predictions[-1].item()
    last_token_weights = mlp_neuron_weights[last_token_idx]
    last_token_mlp_neurons = mlp_neurons[-1]
    last_token_neuron_importance = last_token_weights * last_token_mlp_neurons

    # Calculate the importance as a percentage
    importance_percentage = last_token_neuron_importance / last_token_neuron_importance.sum()
    
    # Add to the results
    prompts_neuron_importance[prompt] = importance_percentage.cpu().numpy()

In [9]:
importance = pd.DataFrame(prompts_neuron_importance)
importance["average"] = importance.sum(axis = 1)/len(importance.columns)

# Format and sort
importance.index.name = "neuron"
importance = importance.sort_values(by="average", ascending=False)

importance.head(10)

,<h1>Title</,<b>Some bold text</,<p>An interesting paragraph</,<table><tr><th>Model name</,<li>List item</,average
neuron,,,,,,
3444,0.096613,0.075791,0.110834,0.096130,0.183113,0.112496
1785,0.101765,0.238890,0.040109,0.017412,0.027491,0.085133
2538,0.024398,0.029209,0.033149,0.034083,0.064888,0.037146
3810,0.051312,0.014371,0.056061,0.015946,0.045200,0.036578
733,0.059635,0.013014,0.009346,0.095818,0.003096,0.036182
1292,0.016477,0.031212,0.018242,0.060624,0.017517,0.028814
1931,0.064833,0.029711,0.038862,0.001635,0.006584,0.028325
3369,0.005495,0.009700,0.039516,-0.001043,0.080424,0.026818
2844,0.059883,0.016307,0.021545,0.017598,0.007661,0.024599


From these results it's clear that a few neurons are important for working out how to close html tags (e.g. `3444`).

##### Visualising key neurons

We'll now run the model with a larger chunk of html, and see which parts trigger these neurons most.

In [10]:
# Create the prompt
prompt = """<html>
  <head>
    <title>Dubious HTML Skills</title>
  </head>
  <div id="heading">
    <h1>
      Dubious HTML Skills
    </h1>
    <p class="subtitle" style="color: red;">
      By Alan Cooney
    </p>
  </div>
  <hr/><!-- Self closing tag to add confusion -->
  <div id="main">
    <h2>
      An unordered list
    </h2>
    <div>
      <ul>
        <li>List item</li>
        <li>Another interesting thing</li>
      </ul>
      <h2>
        An ordered list
      </h2>
      <ol>
         <li>List item</li>
        <li>Another interesting thing</li>
      </ol>
    </div>
  </div>
</html>"""

# Convert to tokens
tokens = model.to_tokens(prompt)

# Setup the cache
model.reset_hooks()
cache = {}
model.cache_all(cache)

# Get the logits
logits = model(prompt, "logits")[0] # [225, 50278]

# Get the neurons
mlp_neurons = cache[f'blocks.{layer}.mlp.hook_post_ln'][0] # [225, 4096]
mlp_neurons.size()

torch.Size([223, 4096])

### Visualising Neuron Activations

We can now run a larger amount of html through the model, and see which parts of the text activate the most important neurons.

#### CSS

In [11]:
%%html
<style>
    /* Container */
    .outerBlock {
        position: relative;
        display: inline-block;
        width: auto;
        padding: 0 5px;
        margin: 1px 1px 1px -2px;
        color: #fff;
    }

    /* Tooltip text */
    .outerBlock .tooltiptext {
      visibility: hidden;
      width: auto;
      background-color: black;
      color: #fff;
      text-align: center;
      padding: 5px 0;
      border-radius: 6px;

      /* Position the tooltip text - see examples below! */
      position: absolute;
      z-index: 1;
    }

    /* Show the tooltip text when you mouse over the tooltip container */
    .outerBlock:hover .tooltiptext {
      visibility: visible;
    }
    </style>

#### Neuron visualisation tool

In [12]:
def text_to_token_strings(text: str) -> list[str]:
    # Extremely hacky function to convert text into a list of each token (as text, not as a token index)
    return model.tokenizer.batch_decode(model.tokenizer.encode(text), clean_up_tokenization_spaces=False)

# Show as a dataframe 
mlp_neurons_df = pd.DataFrame(mlp_neurons.cpu())
mlp_neurons_df["token"] = pd.Series(text_to_token_strings(prompt))
mlp_neurons_df.set_index(["token"], inplace=True)

# Set sum to 1
for col in mlp_neurons_df.columns:
    mlp_neurons_df[col] = mlp_neurons_df[col]/sum(mlp_neurons_df[col])
                                                  
mlp_neurons_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
token,,,,,,,,,,,,,,,,,,,,,
<,0.003533,-0.016636,-0.379564,0.007189,0.087223,0.039597,-0.040201,0.023537,-0.003685,0.019652,...,-0.036820,0.020108,-0.005346,0.026405,0.007802,0.041106,0.009140,-0.141922,-0.001755,-0.195398
html,-0.005276,-0.013208,0.047514,0.003997,0.013514,-0.036720,0.053040,0.003017,-0.001693,0.004344,...,-0.001731,0.006020,-0.002687,0.004635,0.003280,0.002692,0.003184,0.001855,0.001017,0.088594
>,-0.007486,0.003872,-0.032171,0.006462,0.019947,0.000036,0.006352,0.004209,-0.002522,0.006279,...,0.002759,0.000760,-0.002943,0.010412,0.004810,0.005146,0.006146,0.008372,0.005332,-0.054785
\n,-0.006576,-0.007965,-0.007482,0.004003,0.022403,-0.012548,0.024856,0.004416,-0.001774,0.001182,...,0.033956,0.007009,-0.001586,0.011287,0.003906,0.000090,0.005694,-0.004357,-0.002344,-0.043281
,-0.011053,0.009076,-0.132645,0.002152,0.028602,0.009219,-0.003309,0.001926,0.001182,0.005760,...,-0.001476,0.009297,-0.002435,0.016189,0.004565,0.010776,0.005906,0.012155,-0.004963,-0.050460


In [13]:
def visualise_neuron_activation(prompt: list[str], hook_cache: np.ndarray, neuron_index: int) -> None:
    """Visualise the activation of a specific neuron, for an input of text
    
    Args:
        prompt (list[str]): The prompt as a list of string tokens (as text not as indicies)
        hook_cache (np.ndarray): The cache from a specific hook 
            [ number_of_tokens x number_of_neurons ]
        neuron_index (int): The index of the neuron that we're considering
    """
    
    # Get the activations as a percentage
    # Note this will sum to 1, but some neurons will be negative
    activations = hook_cache[:, neuron_index]
    max = activations.max().item()
    min = activations.min().item()
    
    # Print them
    rendered_tokens = []
    for idx, token in enumerate(prompt):
              
        # Handle new lines
        if token == '\n':
            rendered_tokens.append('<br/>')
        
        # Handle spaces
        elif re.match(r"^ *$", token):
            rendered_tokens.append(token.replace(" ", "&nbsp;"))
        
        # Otherwise render the token
        else:
            activation = activations[idx].item()
        
            green = int((activation /(max - min)) * 100 + 125) if activation > 0 else 125
            red = int((activation / (max - min)) * 100 + 125) if activation < 0 else 125
            color = (red, green, 125)

            block = f"""<div style='background: rgb{color}' class="outerBlock">
                {html.escape(token)}
                <span class="tooltiptext">{html.escape(token)} : {round(activation * 100 / (max - min), 1)}%</span>
            </div>"""
            
            rendered_tokens.append(block)
    
    return HTML("<div>" + "\n".join(rendered_tokens) + "</div>")

In [14]:
def visualise_neurons_activations(prompt: list[str], hook_cache: np.ndarray, default_neuron: int) -> None:
    
    number_of_neurons = hook_cache.size()[1]
    
    neuron_dropdown = ipywidgets.Dropdown(
        options=[i for i in range(number_of_neurons)],
        value=default_neuron,
        description='Neuron:',
    )

    display(neuron_dropdown)

    out = ipywidgets.Output()
    display(out)

    with out:
        html_output = visualise_neuron_activation(prompt, hook_cache, default_neuron)
        display(html_output)

    def neuron_dropdown_onchange(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with out:
                clear_output()
                html_output = visualise_neuron_activation(prompt, hook_cache, change['new'])
                display(html_output)

    neuron_dropdown.observe(neuron_dropdown_onchange)

In [15]:
visualise_neurons_activations(text_to_token_strings(prompt), mlp_neurons.cpu(), 3444)

Dropdown(description='Neuron:', index=3444, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…

Output()